In [5]:
import pandas as pd
import numpy as np
import duckdb
import http.client
from datetime import datetime, timedelta
import functions.fbref_functions as func
import tempfile
from dotenv import load_dotenv
import os
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [6]:
## Load dotenv variables
load_dotenv()

#Initialize duckdb with postgres connector    
cursor = duckdb.connect()
cursor.sql("INSTALL postgres;")
cursor.sql("LOAD postgres;")
cursor.sql(f"""
           ATTACH 'dbname=football user={os.getenv("PG_SPORTANALYTICS_USER")} password={os.getenv("PG_SPORTANALYTICS_PASSWORD")} 
           host={os.getenv("PG_SPORTANALYTICS_HOST")}' AS postgres_db (TYPE POSTGRES);
           """)

In [12]:
fixture_url = 'https://fbref.com/en/comps/22/schedule/Major-League-Soccer-Scores-and-Fixtures'
fixtures = pd.read_html(fixture_url, extract_links='body')[0]
fixtures_exp = fixtures.explode(list(fixtures.columns))
fixtures_exp['Score'] = fixtures_exp['Score'].replace('', np.nan)
fixtures_exp['Score'] = fixtures_exp['Score'].replace('Score', np.nan)
fixtures_exp.dropna(subset=['Score'], inplace=True)
fixtures = fixtures_exp.groupby(fixtures_exp.index).first()

# Use duplicated row to define team id of home and away
# And define the url and the match id
id_columns = fixtures_exp.groupby(fixtures_exp.index).last()
fixtures
# [x[0].split('/en/matches/')[1] for x in id_columns['Match Report'].str.rsplit('/', n=1)]

,Round,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
0,Regular Season,1,Wed,2024-02-21,20:00,Inter Miami,1.4,2–0,0.8,Real Salt Lake,"21,137",Chase Stadium,,Match Report,
1,Regular Season,1,Sat,2024-02-24,13:45,LAFC,1.5,2–1,1.8,Seattle Sounders FC,"22,214",BMO Stadium,,Match Report,
2,Regular Season,1,Sat,2024-02-24,14:00,Columbus Crew,1.8,1–0,0.5,Atlanta Utd,"20,406",Lower.com Field,,Match Report,
3,Regular Season,1,Sun,2024-02-25,14:30,FC Cincinnati,0.9,0–0,0.5,Toronto FC,"25,513",TQL Stadium,,Match Report,
4,Regular Season,1,Sun,2024-02-25,16:00,Nashville SC,0.1,0–0,1.4,NY Red Bulls,"30,109",Geodis Park,,Match Report,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,Round One,,Sun,2024-11-03,17:45,Vancouver W'caps,1.3,3–0,1.2,LAFC,,BC Place Stadium,Rosendo Mendoza,Match Report,
554,Round One,,Sat,2024-11-09,16:00,FC Cincinnati,1.2,(5) 0–0 (6),0.3,NYCFC,,TQL Stadium,Drew Fischer,Match Report,
555,Round One,,Sat,2024-11-09,18:00,Orlando City,2.4,(4) 1–1 (1),0.6,Charlotte,"20,021",Inter&Co Stadium,Ismir Pekmic,Match Report,
556,Round One,,Fri,2024-11-08,19:30,LAFC,0.8,1–0,0.6,Vancouver W'caps,,BMO Stadium,Joe Dickerson,Match Report,


In [7]:
competition_url = 'https://fbref.com/en/comps/22/history/Major-League-Soccer-Seasons'
fb_stats = func.fbrefStats(competition_url)
df = fb_stats.get_fixtures()

IndexError: list index out of range

In [3]:
# df.head()
# Define update set for manual update if some data is missing
df.head()

NameError: name 'df' is not defined

In [152]:
match_updates = cursor.sql("""  
                            SELECT * 
                            FROM df 
                            WHERE match_id NOT IN (SELECT match_id FROM postgres_db.gk_stats)
                            OR match_id NOT IN (SELECT match_id FROM postgres_db.shots);
                            """).to_df()
# Reduce input dataframe to max 6 matches, because of Request limits
match_updateset = match_updates.iloc[:6]
match_updateset

,round,wk,day,date,time,home,xg,score,xg.1,away,...,referee,match_report,notes,url,match_id,home_id,away_id,competition,season,inserted_timestamp
0,Regular Season,6,Fri,2024-10-04,20:30,Augsburg,1.3,2–1,0.7,Gladbach,...,Florian Badstübner,Match Report,,https://fbref.com/en/matches/dd751497/Augsburg...,dd751497,0cdc4311,32f3ee20,Fußball-Bundesliga,2024-2025,2024-10-05 07:36:21.222476


In [56]:
table_name = 'fixtures'
matches_updates = cursor.sql(f"SELECT * FROM df WHERE match_id NOT IN (SELECT match_id FROM postgres_db.{table_name});").df()
cursor.sql(f"INSERT INTO postgres_db.{table_name} SELECT * FROM df WHERE match_id NOT IN (SELECT match_id FROM postgres_db.{table_name});")

# Create Team Table

In [58]:
teams = fb_stats.get_teams(df)

In [59]:
table_name = 'teams'
cursor.sql(f"INSERT INTO postgres_db.{table_name} SELECT * FROM teams WHERE team_id NOT IN (SELECT team_id FROM postgres_db.{table_name});")

# Create Scores Table

In [60]:
scores = fb_stats.transform_scores(df)

In [61]:
table_name = 'scores'
cursor.sql(f"INSERT INTO postgres_db.{table_name} SELECT * FROM scores WHERE match_id NOT IN (SELECT match_id FROM postgres_db.{table_name});")

# Get Match Details

In [133]:
match_details = fb_stats.get_match_details(match_updateset)

ValueError: Length mismatch: Expected axis has 24 elements, new values have 13 elements

In [126]:
match_details[0]

,minute,player,squad,xg,psxg,outcome,distance,bodypart,notes,assist_player1,assist1,assist_player2,assist2,team_id,match_id,goalkeeper,competition,season
0,6,Ruben Vargas,Augsburg,0.06,0.00,Off Target,22,Left Foot,,Maximilian Bauer,Pass (Live),Ruben Vargas,Pass (Live),0cdc4311,72461171,Robin Zentner,Fußball-Bundesliga,2024-2025
1,13,Armindo Sieb,Mainz 05,0.25,0.51,Goal,9,Head,,Nadiem Amiri,Pass (Live),Phillipp Mwene,Pass (Dead),a224b06a,72461171,Nediljko Labrović,Fußball-Bundesliga,2024-2025
2,15,Jonathan Burkardt,Mainz 05,0.08,0.32,Goal,11,Head,,Phillipp Mwene,Pass (Live),Jonathan Burkardt,Pass (Live),a224b06a,72461171,Nediljko Labrović,Fußball-Bundesliga,2024-2025
3,22,Jonathan Burkardt,Mainz 05,0.09,0.00,Off Target,24,Right Foot,,Armindo Sieb,Pass (Live),Phillipp Mwene,Pass (Live),a224b06a,72461171,Nediljko Labrović,Fußball-Bundesliga,2024-2025
4,25,Keven Schlotterbeck,Augsburg,0.09,0.13,Goal,8,Left Foot,,Jeffrey Gouweleeuw,Pass (Live),Marius Wolf,Pass (Dead),0cdc4311,72461171,Robin Zentner,Fußball-Bundesliga,2024-2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,67,Tom Bischof,Hoffenheim,0.03,0.00,Blocked,28,Left Foot,,Marius Bülter,Pass (Live),Kevin Akpoguma,Pass (Live),033ea6b8,204b5cf0,Frederik Rønnow,Fußball-Bundesliga,2024-2025
71,67,Marius Bülter,Hoffenheim,0.07,0.62,Goal,15,Right Foot,,Marius Bülter,Take-On,Tom Bischof,Pass (Live),033ea6b8,204b5cf0,Frederik Rønnow,Fußball-Bundesliga,2024-2025
72,79,Diogo Leite,Union Berlin,0.02,0.00,Blocked,14,Head,,László Bénes,Pass (Dead),Yorbe Vertessen,Fouled,7a41008f,204b5cf0,Oliver Baumann,Fußball-Bundesliga,2024-2025
73,90,László Bénes,Union Berlin,0.05,0.00,Off Target,28,Left Foot,Free kick,András Schäfer,Fouled,None,,7a41008f,204b5cf0,Oliver Baumann,Fußball-Bundesliga,2024-2025


# Goalkeeper Match Stats

In [127]:
gk_stats = match_details[1]

In [128]:
gk_stats

,player,age,min,shots,goals,saves,save_perc,psxg,launch_completion,launch_attempts,...,goalkicks_average_length,crosses,crosses_stopped,crosses_stopped_percentage,actions_outside_penaltyarea,actions_average_distance,match_id,competition,season,team_id
0,Nediljko Labrović,24-346,90,3,3,0,0.0,1.8,3,9,...,47.3,4,0,0.0,2,19.7,72461171,Fußball-Bundesliga,2024-2025,0cdc4311
1,Robin Zentner,29-328,90,7,2,5,71.4,2.6,9,28,...,53.4,41,3,7.3,2,11.4,72461171,Fußball-Bundesliga,2024-2025,a224b06a
2,Kevin Müller,33-190,90,5,3,2,40.0,2.5,7,11,...,40.1,13,0,0.0,2,16.0,157d560f,Fußball-Bundesliga,2024-2025,18d9d2a7
3,Noah Atubolu,22-119,90,1,0,1,100.0,0.1,6,18,...,29.8,14,0,0.0,1,11.8,157d560f,Fußball-Bundesliga,2024-2025,a486e511
4,Frederik Rønnow,32-048,90,3,1,2,66.7,0.8,2,9,...,69.3,18,0,0.0,1,20.5,204b5cf0,Fußball-Bundesliga,2024-2025,7a41008f
5,Oliver Baumann,34-111,90,2,2,0,0.0,1.6,4,11,...,65.0,9,1,11.1,2,20.0,204b5cf0,Fußball-Bundesliga,2024-2025,033ea6b8


In [81]:
updates = cursor.sql(f"SELECT * FROM gk_stats WHERE match_id NOT IN (SELECT match_id FROM postgres_db.gk_stats);").to_df()
cursor.sql("SELECT * FROM updates;")

┌────────┬───────┬───────┬───────┬───────┬───┬──────────────────────┬──────────┬─────────────┬────────┬─────────┐
│ player │  age  │  min  │ shots │ goals │ … │ actions_average_di…  │ match_id │ competition │ season │ team_id │
│ int32  │ int32 │ int32 │ int32 │ int32 │   │        int32         │  int32   │    int32    │ int32  │  int32  │
├────────┴───────┴───────┴───────┴───────┴───┴──────────────────────┴──────────┴─────────────┴────────┴─────────┤
│                                                    0 rows                                                     │
└───────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [129]:
table_name = 'gk_stats'
# cursor.sql(f"CREATE TABLE IF NOT EXISTS postgres_db.{table_name} AS SELECT * FROM gk_stats;")
cursor.sql(f"INSERT INTO postgres_db.{table_name} SELECT * FROM gk_stats WHERE match_id NOT IN (SELECT match_id FROM postgres_db.{table_name});")

# Shot Stats

In [130]:
shots = match_details[0]

In [67]:
shots.head()

,minute,player,squad,xg,psxg,outcome,distance,bodypart,notes,assist_player1,assist1,assist_player2,assist2,team_id,match_id,goalkeeper,competition,season
0,1,Theodore Ku-Dipietro,D.C. United,0.07,0.10,Saved,22,Right Foot,,Christian Benteke,Pass (Live),Theodore Ku-Dipietro,Pass (Live),44117292,035907cf,Sean Johnson,Major League Soccer,2024
1,17,Lucas Bartlett,D.C. United,0.06,0.00,Off Target,10,Right Foot,Volley,Mateusz Klich,Pass (Dead),Theodore Ku-Dipietro,Fouled,44117292,035907cf,Sean Johnson,Major League Soccer,2024
2,24,Lorenzo Insigne,Toronto FC,0.04,0.00,Off Target,23,Right Foot,Volley,Matthew Longstaff,Pass (Live),Richie Laryea,Pass (Live),130f43fa,035907cf,Alex Bono,Major League Soccer,2024
3,26,Theodore Ku-Dipietro,D.C. United,0.14,0.53,Saved,16,Left Foot,,Jared Stroud,Pass (Live),None,,44117292,035907cf,Sean Johnson,Major League Soccer,2024
4,26,Derrick Etienne,Toronto FC,0.27,0.20,Saved,12,Right Foot,,Richie Laryea,Pass (Live),Sean Johnson,Pass (Live),130f43fa,035907cf,Alex Bono,Major League Soccer,2024


In [131]:
table_name = 'shots'
# cursor.sql(f"CREATE TABLE IF NOT EXISTS postgres_db.{table_name} AS SELECT * FROM shots;")
cursor.sql(f"INSERT INTO postgres_db.{table_name} SELECT * FROM shots WHERE match_id NOT IN (SELECT match_id FROM postgres_db.{table_name});")

# Reserve / Development Code

In [162]:
for i, row in df.loc[:1].iterrows():
    # Declare variables match_id and url
    match_id = df['match_id'][i]
    url = df['url'][i]
    home_team = df['home_id'][i]

    # Read html output from match url
    html_output = pd.read_html(url, extract_links='body')


    ### Extract goalkeeper statistics from both goalkeeper
    gk_columns = ['player', 'age', 'min', 'shots', 'goals',
    'saves', 'save_perc', 'psxg', 'launch_completion', 
    'launch_attempts', 'launch_comp_percentage', 
    'pass_attempt', 'throws', 'launch_percentage', 
    'pass_average_length', 'goalkicks', 'goalkicks_launched_percentage',
    'goalkicks_average_length', 'crosses', 'crosses_stopped', 
    'crosses_stopped_percentage', 'actions_outside_penaltyarea', 'actions_average_distance']
    
    # Create home goalkeeper dataframe
    gk1_exp = html_output[9].explode(list(html_output[9].columns)).drop(["('Unnamed: 1_level_0',   'Nation')"], axis=1, errors='ignore')
    gk1_output = gk1_exp.groupby(gk1_exp.index).first()
    
    # Create away goalkeeper dataframe
    gk2_exp = html_output[16].explode(list(html_output[16].columns)).drop(["('Unnamed: 1_level_0', 'Nation')"], axis=1, errors='ignore')
    gk2_output = gk2_exp.groupby(gk2_exp.index).first()

    # # Combine home and away GK dataframes
    gk_all_output = pd.concat([gk1_output, gk2_output])
    # gk_all_output = gk_all_output.set_axis(gk_columns, axis=1)

In [55]:
gk_all_output.columns = gk_all_output.columns.map(lambda x: x[1])
gk_all_output.drop(columns=['Nation']).reset_index().drop(columns='index')

In [59]:
gk_all_output.drop(columns=['Nation']).reset_index().drop(columns='index')
# gk_all_output

,Player,Age,Min,SoTA,GA,Saves,Save%,PSxG,Cmp,Att,...,Launch%,AvgLen,Att,Launch%,AvgLen,Opp,Stp,Stp%,#OPA,AvgDist
0,Hugo Lloris,37-060,90,7,1,6,100.0,1.7,6,20,...,48.4,41.9,8,62.5,42.0,13,1,7.7,0,7.0
1,Andrew Thomas,25-176,90,6,2,4,66.7,1.0,5,11,...,35.7,31.7,5,20.0,27.8,12,1,8.3,0,13.2


In [297]:
cursor.sql(f"SELECT * FROM df WHERE match_id NOT IN (SELECT match_id FROM postgres_db.shots);").df()

,round,wk,day,date,time,home,xg,score,xg.1,away,...,referee,match_report,notes,url,match_id,home_id,away_id,competition,season,inserted_timestamp


In [11]:
gk_columns = ['player', 'age', 'min', 'shots', 'goals',
'saves', 'save_perc', 'psxg', 'launch_completion', 
'launch_attempts', 'launch_comp_percentage', 
'pass_attempt', 'throws', 'launch_percentage', 
'pass_average_length', 'goalkicks', 'goalkicks_launched_percentage',
'goalkicks_average_length', 'crosses', 'crosses_stopped', 
'crosses_stopped_percentage', 'actions_outside_penaltyarea', 'actions_average_distance']

gk_stats = pd.DataFrame(columns=gk_columns)

shot_columns = ['minute', 'player', 'squad', 'xg', 'psxg', 'outcome', 
                'distance', 'bodypart', 'notes', 'assist_player1', 
                'assist1', 'assist_player2', 'assist2', 'team_id', 
                'match_id', 'goalkeeper', 'competition', 'season']



In [12]:
gk_stats

,player,age,min,shots,goals,saves,save_perc,psxg,launch_completion,launch_attempts,...,launch_percentage,pass_average_length,goalkicks,goalkicks_launched_percentage,goalkicks_average_length,crosses,crosses_stopped,crosses_stopped_percentage,actions_outside_penaltyarea,actions_average_distance


In [31]:
html_output = pd.read_html('https://fbref.com/en/matches/4e23f1b6/St-Pauli-RB-Leipzig-September-22-2024-Bundesliga')

In [28]:
shot_columns = ['minute', 'player', 'squad', 'xg', 'psxg', 
                'outcome', 'distance', 'bodypart', 'notes', 
                'assist_player1', 'assist1', 'assist_player2', 'assist2']
shot_exp = html_output[-3].explode(list(html_output[-3].columns))
shot_exp[shot_exp.columns[1]] = shot_exp[shot_exp.columns[1]].replace('', np.nan)
shot_exp[shot_exp.columns[-2]] = shot_exp[shot_exp.columns[-2]].replace('', np.nan)
shot_exp[shot_exp.columns[-4]] = shot_exp[shot_exp.columns[-4]].replace('', np.nan)
shot_exp.dropna(subset=[shot_exp.columns[1]], inplace=True)
shot_output = shot_exp.groupby(shot_exp.index).first()
shot_output = shot_output.set_axis(shot_columns, axis=1)
shot_output.dropna(subset=['minute'], inplace=True)
shot_output = shot_output.set_axis(shot_columns, axis=1)
shot_output.dropna(subset=['minute'], inplace=True)
shot_output

,minute,player,squad,xg,psxg,outcome,distance,bodypart,notes,assist_player1,assist1,assist_player2,assist2
0,3,Manolis Saliakas,St. Pauli,0.10,0.00,Blocked,14.0,Left Foot,None,Manolis Saliakas,Take-On,Johannes Eggestein,Pass (Live)
1,14,Yussuf Poulsen,RB Leipzig,0.16,0.00,Blocked,12.0,Right Foot,None,David Raum,Pass (Live),Xavi Simons,Pass (Live)
2,17,Elias Saad,St. Pauli,0.04,0.02,Saved,16.0,Right Foot,None,Philipp Treu,Pass (Live),Eric Smith,Pass (Live)
3,17,Carlo Boukhalfa,St. Pauli,0.03,0.00,Off Target,11.0,Head,None,Eric Smith,Pass (Dead),Elias Saad,Shot
4,18,Jackson Irvine,St. Pauli,0.23,0.00,Off Target,6.0,Head,None,Hauke Wahl,Pass (Live),Carlo Boukhalfa,Shot
5,18,Johannes Eggestein,St. Pauli,0.03,0.06,Saved,30.0,Right Foot,None,Carlo Boukhalfa,Pass (Live),Johannes Eggestein,Pass (Live)
6,19,Christoph Baumgartner,RB Leipzig,0.12,0.00,Off Target,6.0,Head,None,David Raum,Pass (Live),Nicolas Seiwald,Pass (Live)
7,22,Karol Mets,St. Pauli,0.01,0.00,Off Target,12.0,Head,None,Karol Mets,Pass (Live),Eric Smith,Pass (Live)
8,35,Jackson Irvine,St. Pauli,0.03,0.00,Off Target,11.0,Head,None,Manolis Saliakas,Pass (Live),Elias Saad,Pass (Live)
9,36,Carlo Boukhalfa,St. Pauli,0.07,0.00,Blocked,15.0,Left Foot,Volley,Johannes Eggestein,Pass (Live),Carlo Boukhalfa,Pass (Live)


In [27]:
import glob
directory = '/Users/matthiashugli/kDrive/bucket/hockey/nhl-goalies-2025/*.parquet'

df = pd.DataFrame()
for filename in glob.glob(directory):
    file = pd.read_parquet(filename)
    df = pd.concat([df, file])


In [28]:
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))

556
467


In [ ]:
table_name = 'postgresdb.'
cursor.sql(f"DELETE FROM {table_name};")
cursor.sql(f"INSERT INTO {table_name} SELECT * FROM th_presence;")
row_amount = cursor.sql(f"SELECT count(*) FROM {table_name};")

In [5]:
url = 'https://www.hockey-reference.com/leagues/NHL_2025_goalies.html'
goalie_stats = pd.read_html(url)[0]
goalie_stats.columns = [x[1] for x in goalie_stats.columns]
goalie_stats = goalie_stats.iloc[:, :-5]

In [22]:
goalie_stats1 = pd.read_parquet('/Users/matthiashugli/kDrive/bucket/hockey/nhl-goalies-2025/nhl_goalie_stats_2024-10-28.parquet').iloc[:, :22]
goalie_stats2 = pd.read_parquet('/Users/matthiashugli/kDrive/bucket/hockey/nhl-goalies-2025/nhl_goalie_stats_2024-11-10.parquet').iloc[:, :22]
goalie_stats = pd.concat([goalie_stats1, goalie_stats2])

In [25]:
if 'Tm' in goalie_stats.columns:
    goalie_stats = goalie_stats.rename(columns={'Tm': 'Team'})

In [27]:
goalie_stats.columns

Index(['Rk', 'Player', 'Age', 'Team', 'GP', 'GS', 'W', 'L', 'T/O', 'GA', 'SA',
       'SV', 'SV%', 'GAA', 'SO', 'GPS', 'MIN', 'QS', 'QS%', 'RBS', 'GA%-',
       'GSAA', 'Team', 'Pos', 'Shots'],
      dtype='object')